In [15]:
import numpy as np
import os
import re

import multiprocess as mp
import warnings
warnings.filterwarnings("ignore")

import plotly.graph_objs as go

from trProcess import *

In [16]:
def ProcessBaseline (conf):
    exps, expconf, tracenum, indir, name, baseline = conf
    minApp, jfi = [], []
    for exp in exps:
        if not re.search (expconf, exp):
            continue
        minAppTrace, jfiTrace = [], []
        for traceid in range(tracenum):
            trace = "maxwell_20.0_0.01_300_%d.pitree-trace" % traceid
            conf = os.path.join (indir, exp, baseline, trace)
                
            _, _, _, bws, bwSeries = bwTraceProcessor (os.path.join (conf, 'bw.tr'), startTime=100000, endTime=200000)
            _, _, avgDelay, allDelay = appTraceProcessor (os.path.join (conf, 'app.tr'), startTime=100000, duration=100000)
            curMinAppFlows = np.inf
            for port, appDelay in avgDelay.items():
                curMinAppFlows = min(curMinAppFlows, appDelay)
            minAppTrace.append(curMinAppFlows)
            bws = np.array(list(bws.values()))
            jfiTrace.append(pow(np.nansum(bws),2) / (bws.size * np.nansum(np.square(bws))))
            print ("%.2f\t%.3f\t%s" % (avgDelay[10000] + 20, np.average (jfiTrace), name))
            print ("%.2f\t%.3f\t%s" % (avgDelay[20000] + 20, np.average (jfiTrace), name))
        minApp.append(np.average(minAppTrace))
        jfi.append(np.average(jfiTrace))
    return go.Scatter(
        x=[np.average (minApp)],
        y=[np.average (jfi)],
        mode="markers+text",
        text=name,
        name=name,
        textposition="bottom center"
    )

In [17]:
tracenum = 1
baselinelist = {
    "FQ": "FqCoDel_0_500ms500", 
    "FIFO": "PfifoFast_0_500",
    "HHF": "Hhf_0_500", 
    "Confucius": "Auto_0_1.00_ExpClass4.00",
    # "CBQ": "Dwrr_9_1.00",
    # "StrictPriority" : "StrictPriority_9_1000",
}

expconfs = [
    "^4560_Video1_Video1_Video1_Video1$",
]
indir = "ns-3.34/logs/"
exps = os.listdir (indir)
for expconf in expconfs:
    # pool = mp.Pool (4)
    # data1 = pool.map (ProcessBaseline, [(exps, expconf, tracenum, indir, name, baseline) for name, baseline in baselinelist.items()])
    # pool.close ()
    # pool.join ()
    data1 = list (map (ProcessBaseline, [(exps, expconf, tracenum, indir, name, baseline) for name, baseline in baselinelist.items()]))

    fig = go.Figure (data=data1)
    fig.update_layout(
        title=expconf,
        xaxis_title="Minimum average delay",
        yaxis_title="JFI"
    )
    fig.show()

49.84	0.944	FQ
60.65	0.944	FQ
148.94	0.278	FIFO
13203.15	0.278	FIFO
47.27	0.640	HHF
65.67	0.640	HHF
49.70	0.985	Confucius
61.79	0.985	Confucius
